# Функции, введение в ООП

В данном случае вас ждут:
- Все виды параметров и вызовов: позиционные, ключевые, `*args`, `**kwargs`, позиционно‑только (`/`)
- Замыкания, области видимости (LEGB), `nonlocal`/`global`
- Значения по умолчанию и «ловушки» (mutable defaults)
- Аннотации типов: `typing.Callable`, `ParamSpec`, `TypeVar`, `overload`, `Protocol`
- Функции как объекты первого класса, высшие функции, каррирование, `functools`
- Декораторы (в т.ч. параметризованные), `wraps`, мемоизация `lru_cache`
- Исключения, контракты и дизайн API; документация (docstring) и `help()`
- Производительность, рекурсия, генераторы vs обычные функции, `yield from`
- Задачи на прокачку уровня собеседований


## 1. Базовый синтаксис `def`

In [1]:
def area(w: float, h: float) -> float:
  """Возвращает площадь прямоугольника"""
  return w * h

print(area(3, 4))
display(help(area))
display(dir(area))

12
Help on function area in module __main__:

area(w: float, h: float) -> float
    Возвращает площадь прямоугольника



None

['__annotations__',
 '__builtins__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__type_params__']

In [2]:
class Pokemon:
  def __init__(self):
    print(self)
    self.a = 12

class Area:
  _obj = None
  def __new__(cls, *args, **kwargs):
    print(cls)
    if Area._obj is None:
      Area._obj = Pokemon()
      return Area._obj
    else:
      return Area._obj



  def __init__(this):
    print(this)
    this.x = 10


  def __call__(self, width: int, height: int):
    return width * height


ar = Area()





<class '__main__.Area'>


In [3]:
type(ar)

__main__.Pokemon

In [4]:
ar2 = Area()


id(ar2), id(ar)

<class '__main__.Area'>


(2630523654832, 2630523654832)

In [5]:
dir(ar.x)

AttributeError: 'Pokemon' object has no attribute 'x'

## Виды параметров в `Python`

Порядок категорий в сигнатуре:  
1) **Позиционные‑только**: `a, b, /`  
2) **Позиционные‑или‑ключевые**: `c, d`  
3) **Перем. позиционные**: `*args`  
4) **Ключевые‑только**: `*, e, f=...`  
5) **Перем. ключевые**: `**kwargs`

**Почему это важно на собеседовании:** корректный дизайн сигнатур API, стабильность и читаемость.

In [ ]:
def f(*lists, **dicts):
  ...


In [ ]:
def demo_params(a, b=10, /, c=20, *args, d, e=50, **kwargs):
    """Пример смешанной сигнатуры.
    a, b — позиционно-только (из-за '/')
    c — позиционный или ключевой
    *args — дополнительные позиционные
    d, e — ключевые-только (из-за '*')
    **kwargs — дополнительные ключевые
    """
    return {
        "a": a, "b": b, "c": c, "args": args, "d": d, "e": e, "kwargs": kwargs
    }

# Валидные вызовы
print(demo_params(1, 2, 3, 4, 5, d=6, e=7, x=8))
print(demo_params(1, 2, d=6))  # c использует значение по умолчанию
# Невалидно: demo_params(a=1, b=2, d=6)  # a,b позиционно-только


Если у нас нет дополнительных позиционных, но хочется иметь именованные

In [ ]:
def demo_params_stars(a, b=10, /, c=20, *, d, e=50, **kwargs):
    """Пример смешанной сигнатуры.
    a, b — позиционно-только (из-за '/')
    c — позиционный или ключевой
    *args — дополнительные позиционные
    d, e — ключевые-только (из-за '*')
    **kwargs — дополнительные ключевые
    """
    return {
        "a": a, "b": b, "c": c,  "d": d, "e": e, "kwargs": kwargs
    }


print(demo_params_stars(10, 12, 13, d=14, j = 15))
print(demo_params_stars)
print(demo_params_stars.__defaults__)

## Распаковка и согласование аргументов

In [ ]:
def pair_sum(x, y): return x + y

t = (3, 5)

d = {
    "y": 10, "x": 7
}


print(pair_sum(*t))

print(pair_sum(**d))

Все в месте

In [ ]:
def function(a, /, b, *args, c, **kw):
  return a, b, args, c, kw


print(function(1, 2, *[3, 4], **{"c": 5, "d": 6} ))

## Напоминание: Значения по умолчанию и "ловушка" изменяемости

In [ ]:

def append_item_bad(item, container=[]):  # анти‑паттерн
    container.append(item)
    return container

print(append_item_bad(1))
print(append_item_bad(2))  # накапливается в общем списке!

def append_item_good(item, container=None):
    container = container or []
    # if container is None:
    #     container = []
    container.append(item)
    return container

print(append_item_good(1))
print(append_item_good(2))

## Области видимости

- **L**ocal → **E**nclosing → **G**lobal → **B**uiltins  
- `nonlocal` для замыкания, `global` для модуля. Избегайте `global` в дизайне API.

In [ ]:
x = "global"

def outer():
    x = "enclosing"
    def inner():
        nonlocal x
        x = "changed in inner"
        return x
    inner()
    return x

print(outer())  # 'changed in inner'
print(x)        # 'global'

## Замыкания и функции высшего порядка

Функция — объект первого класса: можно хранить в переменных, передавать аргументом, возвращать из функции.

In [ ]:
def make_multiplier(k: int):
    def mul(x: int) -> int:
        return k * x
    return mul

double = make_multiplier(2)
triple = make_multiplier(3)
print(double(10), triple(10))
for closure in double.__closure__:
  print("closure {0}, content: {1}".format(closure, closure.cell_contents))  # захваченное значение k

- __closure__ → None, если функция ничего не замыкает.
- cell_contents — это ссылка на живой объект, а не копия!
- Если переменная изменится в nonlocal, cell_contents тоже изменится.
- Используется редко в продакшене, но очень часто спрашивается на собеседованиях:
«Объясните, как работает замыкание и где хранится значение захваченной переменной?»

In [ ]:
def outer(a, b):
    def inner(x):
        return a * x + b
    return inner

f = outer(2, 3)
for i, cell in enumerate(f.__closure__):
    print(f"closure[{i}] =", cell.cell_contents)

### Анонимные функции `lambda`
lambda` — компактный способ объявить функцию *на лету*, без `def`. Это полноценный объект-функция с ограничением: тело — **одно выражение**.

#### Когда уместно
- короткие одноразовые функции как аргументы (`key=`, `map`, `filter`);
- простая функциональная композиция;
- фабрики функций внутри других функций.

> На собеседованиях важно уметь объяснить разницу `lambda` vs `def`, показать примеры с `sorted(key=...)`, `map`, `filter`, и продемонстрировать замыкание внутри `lambda`.

In [ ]:
# Базовый синтаксис и эквивалентность с def
def square(x): return x ** 2
square2 = lambda x: x ** 2

print(square(5), square2(5))
print(type(square2), square2.__name__)  # __name__ == "<lambda>"


In [ ]:
# Использование с map / filter / sorted / reduce
from functools import reduce
nums = [1, 2, 3, 4, 5]

print(list(map(lambda x: x**2, nums)))            # квадрат каждого
print(list(filter(lambda x: x % 2 == 0, nums)))   # только чётные
print(sorted(["apple", "pear", "banana"], key=lambda s: len(s)))

# reduce: произведение элементов
prod = reduce(lambda acc, x: acc * x, nums, 1)
print(prod)

In [ ]:
# Lambda и замыкания
def make_power(p):
    return lambda x: x ** p  # захватывает p из внешней области

square = make_power(2)
cube = make_power(3)
print(square(6), cube(6))

# Вложенные lambda
adder = lambda a: (lambda b: a + b)
add5 = adder(5)
print(type(adder))
print(add5(10))


In [ ]:
# Частый паттерн: key-функции
users = [
    {"name": "Alice", "age": 25},
    {"name": "Bob", "age": 18},
    {"name": "Carol", "age": 30},
]
print(sorted(users, key=lambda u: (u["age"], u["name"])))



### Антипаттерны и советы
- Не пихайте сложную логику в `lambda` — лучше `def` с понятным именем и докстрингом.
- Тернарные выражения допустимы, но **добавляйте скобки** для ясности в длинных выражениях.
- Для многократного переиспользования чаще подходит `def` или `functools.partial`.
- Отладка: у `lambda` имя `"<lambda>"`; в логах и трейсах это менее информативно.

### Для интервью
- `lambda` создаёт объект-функцию, у которой есть `__code__`, `__defaults__`, `__closure__`.
- Отличия от `def`: нельзя несколько выражений, `return` не используется (результат — значение выражения).
- Показать пример замыкания с `lambda` и объяснить модель LEGB.


### Задачи на практику
Решите в одну строчку
1) Отсортируйте список строк по: (длина, затем лексикографически в обратном порядке)
2) Верните топ-3 элементов по частоте встречаемости

На подумать

3) Композиция двух функций f∘g с помощью lambda


## Декораторы: базовые и параметризованные

**Декоратор** — это функция (или класс), которая принимает другую функцию (или метод, класс)
и возвращает новую функцию — обёртку с дополнительным поведением.
Другими словами, декоратор — это способ изменить поведение функции, не изменяя её код.
Это чистая реализация принципа Open–Closed (функция открыта для расширения, но закрыта для модификации).

In [ ]:
def log_calls(func):
    def wrapper(*args, **kwargs):
        print(f"[CALL] {func.__name__}{args, kwargs}")
        result = func(*args, **kwargs)
        print(f"[RET]  {func.__name__} -> {result}")
        return result
    return wrapper

@log_calls
def pow_int(x: int, y: int) -> int:
    """power int"""
    return x ** y

f(2, 10)
print(f.__name__, f.__doc__)

In [ ]:
from functools import wraps

def log_calls(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"[CALL] {func.__name__}{args, kwargs}")
        result = func(*args, **kwargs)
        print(f"[RET]  {func.__name__} -> {result}")
        return result
    return wrapper

@log_calls
def pow_int(x: int, y: int) -> int:
    """power int"""
    return x ** y

pow_int(2, 10)
print(pow_int.__name__, pow_int.__doc__)

Пример параметризированного декоратора

In [ ]:
# Параметризованный декоратор
def retry(retries=3):
    def deco(func):
        @wraps(func)
        def wrapper(*args, **kw):
            last = None
            for _ in range(retries):
                try:
                    return func(*args, **kw)
                except Exception as e:
                    last = e
            raise last
        return wrapper
    return deco

@retry(retries=2)
def flaky():
    import random
    if random.random() < 0.7:
        raise RuntimeError("flaky")
    return "ok"

try:
    print(flaky())
except RuntimeError as e:
    print("Failed after retries:", e)


## Как посмотреть внутрь закулисья: `inspect.signature`, `BoundArguments`

Полезно для валидации, роутинга, написания фреймворков.

In [ ]:

import inspect

sig = inspect.signature(demo_params)
print(sig)

ba = sig.bind(1, 2, d=6)
ba.apply_defaults()
print("Bound:", ba.arguments)

# Разбор параметров и их kinds
for name, p in sig.parameters.items():
    print(name, p.kind, p.default is inspect._empty, p.annotation)

## Аннотации типов для функций (интервью‑критично)

- `Callable`, `TypeVar`, `ParamSpec`, `Concatenate`
- Протоколы с `Protocol` (структурная типизация)
- Перегрузка `@overload` для точных сигнатур


In [ ]:
from typing import Callable, TypeVar, ParamSpec, Concatenate, Protocol, overload

T = TypeVar('T')
P = ParamSpec('P')

def apply(func: Callable[[T], T], value: T) -> T:
    return func(value)

print(apply(lambda x: x + 1, 10))

# Хелпер с сохранением параметров и добавлением контекста
def with_logger(func: Callable[P, T]) -> Callable[Concatenate[str, P], T]:
    def wrapper(prefix: str, *args: P.args, **kw: P.kwargs) -> T:
        print(prefix, "→ calling", func.__name__)
        return func(*args, **kw)
    return wrapper

@with_logger
def add(a: int, b: int) -> int: return a + b

print(add("LOG", 2, 3))

# Protocol: любой объект с методом .transform(int)->int подойдёт
class Transformer(Protocol):
    def transform(self, x: int) -> int: ...

class Doubler:
    def transform(self, x: int) -> int: return 2 * x

def use_transformer(t: Transformer, x: int) -> int:
    return t.transform(x)

print(use_transformer(Doubler(), 5))

# overload: разные сигнатуры для одного тела
@overload
def to_list(x: int) -> list[int]: ...
@overload
def to_list(x: str) -> list[str]: ...

def to_list(x):
    return [x]

print(to_list(1), to_list("a"))


## `functools`: `partial`, `lru_cache`, `singledispatch`

In [ ]:

from functools import partial, lru_cache, singledispatch

def mul(a, b): return a * b
times10 = partial(mul, 10)
print(times10(7))

@lru_cache(maxsize=128)
def fib(n: int) -> int:
    if n < 2: return n
    return fib(n-1) + fib(n-2)

print(fib(30), fib.cache_info())

@singledispatch
def dump(value): print("generic:", value)

@dump.register
def _(value: int): print("int:", value)

@dump.register
def _(value: list): print("list:", value)

dump(42); dump([1,2,3]); dump({"a":1})


## Генераторы vs обычные функции, `yield from`
Генератор — это функция, которая не возвращает результат сразу,
а постепенно “выдаёт” значения по мере запроса через yield.

Каждый вызов next() — это возобновление выполнения функции с того места,
где она остановилась на последнем yield.


In [ ]:
def squares(n):
    for i in range(n):
        yield i*i

def chain(*iters):
    for it in iters:
        yield from it

print(list(chain(squares(3), [9, 16])))


## Атрибуты функции: `__defaults__`, `__kwdefaults__`, `__code__`


In [ ]:

def g(a, b=10, *, c=20): pass

print("defaults:", g.__defaults__)
print("kwdefaults:", g.__kwdefaults__)
print("code vars:", g.__code__.co_varnames)
print("argcount:", g.__code__.co_argcount, "kwonly:", g.__code__.co_kwonlyargcount)



## Задачи на прокачку: мини‑задачи уровня собеседования

1. **Safe get**: Напишите функцию `safe_get(d, path, default=None)` которая возвращает `d[a][b][c]...` по списку ключей `path`, не кидая исключение при отсутствии.
2. **Unique key func**: Реализуйте `unique_by(iterable, key)` — возвращает элементы, уникальные по `key`‑функции, сохраняя порядок.
3. **Retry decorator**: Напишите декоратор `retry_backoff(retries, base_delay)` с экспоненциальной паузой `base_delay * 2**attempt`.
4. **Memoize**: Собственная версия `memoize` для чистых функций с ограничением размера кэша.
5. **Flatten**: Функция `flatten` для произвольной вложенности списков/кортежей (только последовательности, строки не трогаем).
